In [1]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [3]:
api_wraper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wraper)
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\material\\GenerativeAI\\langchain\\qna_chatbot_for_edtech\\env\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250))

In [4]:
api_wraper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=255)
axiv=ArxivQueryRun(api_wrapper=api_wraper_arxiv)
axiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=255))

In [5]:
tools=[wiki,axiv]

In [28]:
# custome tools
from langchain_community.document_loaders import WebBaseLoader,WikipediaLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
load_dotenv()
import os



In [40]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
api=os.getenv("GROQ_API_KEY")

In [20]:
embedding=HuggingFaceEmbeddings(model_name="avsolatorio/GIST-Embedding-v0")

e:\material\GenerativeAI\langchain\qna_chatbot_for_edtech\env\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dell\.cache\huggingface\hub\models--avsolatorio--GIST-Embedding-v0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
e:\material\GenerativeAI\langchain\qna_chatbot_for_edtech\env\Lib\site-pac

In [29]:
loader=WikipediaLoader("Virat_Kohli")
docs=loader.load()
docs


[Document(metadata={'title': 'Virat Kohli', 'summary': "Virat Kohli (Hindi pronunciation: [ʋɪˈɾɑːʈ ˈkoːɦli] ; born 5 November 1988) is an Indian international cricketer who plays Test and One Day International (ODI) cricket for the Indian national team. A former captain in all formats, Kohli retired from Twenty20 International (T20I) following India's win at the 2024 T20 World Cup. He is a right-handed batsman and an occasional unorthodox right arm quick bowler. He represents Royal Challengers Bengaluru in the Indian Premier League (IPL) and Delhi in domestic cricket. He holds the record as the highest run-scorer in  IPL, ranks third in T20I, third in ODI, and stands as the fourth-highest in international cricket. He also holds the record for scoring the most centuries in ODI cricket and stands second in the list of most international centuries scored. Kohli is widely regarded as one of the greatest batsmen of all time and the greatest batsman in the modern era. Kohli was a key member 

In [33]:
document=docs[0:2]

In [34]:
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100).split_documents(document)

In [35]:
vectordb=FAISS.from_documents(documents,embedding)

In [36]:
retriever=vectordb.as_retriever()

In [37]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000014772F682D0>)

In [38]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"Virat Kohli","Serach About Virat Kohli")
retriever_tool.name

'Virat Kohli'

In [39]:
tools=[wiki,axiv,retriever_tool]

In [41]:
from langchain_groq import ChatGroq
llm=ChatGroq(api_key=api,model="Llama3-8b-8192")

In [44]:
# prompt template
from langchain import hub
promt=hub.pull("hwchase17/openai-functions-agent")
promt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [47]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm=llm,tools=tools,prompt=promt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_has

In [48]:
# to run the agent we need to create agent executer
from langchain.agents import AgentExecutor
agent_executer=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [50]:
agent_executer.invoke({"input":"Tell me about virat kohli"})



> Entering new AgentExecutor chain...

Invoking: `Virat Kohli` with `{'query': 'Virat Kohli'}`


Virat Kohli (Hindi pronunciation: [ʋɪˈɾɑːʈ ˈkoːɦli] ; born 5 November 1988) is an Indian international cricketer who plays Test and One Day International (ODI) cricket for the Indian national team. A former captain in all formats, Kohli retired from Twenty20 International (T20I) following India's win at the 2024 T20 World Cup. He is a right-handed batsman and an occasional unorthodox right arm quick bowler. He represents Royal Challengers Bengaluru in the Indian Premier League (IPL) and Delhi in domestic cricket. He holds the record as the highest run-scorer in  IPL, ranks third in T20I, third in ODI, and stands as the fourth-highest in international cricket. He also holds the record for scoring the most centuries in ODI cricket and stands second in the list of most international centuries scored. Kohli is widely regarded as one of the greatest batsmen of all time and the greatest batsman

{'input': 'Tell me about virat kohli',
 'output': 'Here is the response:\n\nVirat Kohli is an Indian international cricketer who plays Test and One Day International (ODI) cricket for the Indian national team. He is a right-handed batsman and an occasional unorthodox right arm quick bowler. He represents Royal Challengers Bengaluru in the Indian Premier League (IPL) and Delhi in domestic cricket. He holds the record as the highest run-scorer in IPL, ranks third in T20I, third in ODI, and stands as the fourth-highest in international cricket.'}

In [52]:
agent_executer.invoke({"input":"what is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks withBased on the result of the tool call "call_qcj7", I'd like to continue the conversation.

It seems that machine learning is a field of study that focuses on developing and studying statistical algorithms that can learn from data and generalize to unseen data. This is often used to perform tasks such as pattern recognition, classification, and prediction.

Would you like to know more about the types of machine learning algorithms, or perhaps how machine learning is applied in different industries? Or would you like to ask a more specific question about machine learning?

> Finished chain.


{'input': 'what is machine learning',
 'output': 'Based on the result of the tool call "call_qcj7", I\'d like to continue the conversation.\n\nIt seems that machine learning is a field of study that focuses on developing and studying statistical algorithms that can learn from data and generalize to unseen data. This is often used to perform tasks such as pattern recognition, classification, and prediction.\n\nWould you like to know more about the types of machine learning algorithms, or perhaps how machine learning is applied in different industries? Or would you like to ask a more specific question about machine learning?'}

In [53]:
agent_executer.invoke({"input":"what are two question i asked you ?"})



> Entering new AgentExecutor chain...
I'm happy to help! However, since we just started our conversation, you haven't asked me any questions yet. Please feel free to ask me anything, and I'll do my best to assist you!

> Finished chain.


{'input': 'what are two question i asked you ?',
 'output': "I'm happy to help! However, since we just started our conversation, you haven't asked me any questions yet. Please feel free to ask me anything, and I'll do my best to assist you!"}

In [54]:
agent_executer.invoke({"input":"hey just let me know what's the paper 1706.03762 about ?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on comple
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper authored by eight scientists working at Google, that introduced a new deep learning architecture known as the transformer based on attention mechanThe paper "Attention Is All You Need" by Vaswani et al. (2017) introduced the Transformer model, which is a type of recurrent neural network (RNN) that uses self-attention mechanisms to process input sequences. The Transformer architecture is particularly well-suited for natural language processing tasks, such as machine translati

{'input': "hey just let me know what's the paper 1706.03762 about ?",
 'output': 'The paper "Attention Is All You Need" by Vaswani et al. (2017) introduced the Transformer model, which is a type of recurrent neural network (RNN) that uses self-attention mechanisms to process input sequences. The Transformer architecture is particularly well-suited for natural language processing tasks, such as machine translation and text classification. The paper\'s authors showed that the Transformer model outperformed other state-of-the-art models on a range of machine translation tasks, and it has since become a widely-used and influential architecture in the field of natural language processing.'}